In [7]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

In [10]:
class conv_block(nn.Module):
    def __init__(self, in_size, out_size, kernal = 3, stride = 1):
        super(conv_block, self).__init__()
        
        padding = (kernal-1)//2
        self._conv_pri = nn.Sequential(
            nn.Conv2d(in_size, out_size, kernal, stride, padding),
            nn.BatchNorm2d(out_size),
            nn.Relu(True)
        )
        
    def forward(self, x):
        out = self._conv_pri(x)
        return out
    
class res_block(nn.Module):
    def __init__(self, in_size, out_size, kernal = 3, stride = 1):
        super(conv_block, self).__init__()
        
        padding = (kernal-1)//2
        self._conv1_pri = conv_block(in_size, out_size, kernal, stride, padding)
        self._conv2_pri = conv_block(out_size, out_size, kernal, stride, padding)
        self._relu_pri = nn.Relu()
        
    def forward(self, x):
        out = self._conv1_pri(x)
        out = self._conv2_pri(out)        
        out = out + x
        return out

class colorization_net(nn.Module):
    def __init__(self):
        super(colorization_net, self).__init__()
        
        self._conv1_pri = conv_block(1, 64, 9)
        self._res1_pri = res_block(64, 64)
        self._res2_pri = res_block(64, 64)
        self._res3_pri = res_block(64, 64)
        self._res4_pri = res_block(64, 64)
        self._conv2_pri = nn.Sequential(
            nn.Conv2d(64, 2, 9, 1, (9-1)//2 ),
            nn.Tanh()
        )
        
    def forward(self, x):
        out = self._conv1_pri(x)
        out = self._res1_pri(out)
        out = self._res2_pri(out)
        out = self._res3_pri(out)
        out = self._res4_pri(out)
        out = self.conv2_pri(out)
        out = (out + 1) * 127.5
        
        return out;